In [ ]:
Query FindOrd(code = "") As websys.Query(ROWSPEC = "病区编号:%String:病区编号,病区名称:%String:病区名称,科室编号:%String:科室编号,科室名称:%String:科室名称,病人编号:%String:病人编号,病人姓名:%String:病人姓名,住院号:%String:住院号,住院次数:%String:住院次数,床位名称:%String:床位名称,医嘱编号:%String:医嘱编号,医嘱名称:%String:医嘱名称,医嘱标识:%String:医嘱标识,医嘱类别:%String:医嘱类别,医嘱单位:%String:医嘱单位,医嘱用量:%String:医嘱用量,医嘱执行状态:%String:医嘱执行状态,开嘱医生:%String:开嘱医生,开始时间:%String:开始时间,结束时间:%String:结束时间,医嘱频次:%String:医嘱频次,医嘱嘱托:%String:医嘱嘱托") [ SqlProc ]
{
}

/// debug:	d ##class(%ResultSet).RunQuery("web.DHCDocMain", "FindOrd","ZY034")
/// call web.DHCDocMain_FindOrd()
ClassMethod FindOrdExecute(ByRef qHandle As %Binary, code = "") As %Status
{
	/*
病区编号 Varchar No 若有科室无病区，可不提供。
病区名称 Varchar No
科室编号 Varchar No 若有病区无科室，可不提供。
科室名称 Varchar No
病人编号 Varchar No 患者在院唯一标识号（永久）
病人姓名 Varchar No
住院号 Varchar No
住院次数 Varchar No
床位名称 Varchar No 床位号（1床、2床…）
医嘱编号 Varchar No
医嘱名称 Varchar No 血糖医嘱、用药医嘱
医嘱标识 Varchar No 区分治疗检验药品等
医嘱类别 Varchar No 长期医嘱、临时医嘱
医嘱单位 Varchar Yes
医嘱用量 Varchar Yes
医嘱执行状态 Varchar No 执行、停止、撤销等
开嘱医生 Varchar No
开始时间 datetime No 医嘱开始时间
结束时间 datetime Yes 医嘱结束时间
医嘱频次 Varchar No
医嘱嘱托 Varchar Yes

病区编号:%String:病区编号,病区名称:%String:病区名称,科室编号:%String:科室编号,科室名称:%String:科室名称,病人编号:%String:病人编号,病人姓名:%String:病人姓名,住院号:%String:住院号,住院次数:%String:住院次数,床位名称:%String:床位名称,医嘱编号:%String:医嘱编号,医嘱名称:%String:医嘱名称,医嘱标识:%String:医嘱标识,医嘱类别:%String:医嘱类别,医嘱单位:%String:医嘱单位,医嘱用量:%String:医嘱用量,医嘱执行状态:%String:医嘱执行状态,开嘱医生:%String:开嘱医生,开始时间:%String:开始时间,结束时间:%String:结束时间,医嘱频次:%String:医嘱频次,医嘱嘱托:%String:医嘱嘱托
	*/
	
	s $zt="muse"
	s repid = $i(^CacheTemp)
	s ind = 1
	s qHandle = $lb(0, repid, 0)
	if code = "" {
		s loc = 0
		for {
			s loc = $o(^CTLOC(loc))
			q:loc=""
			s CTLOCDepDR=$p($g(^CTLOC(loc)),"^",19)
    		s:CTLOCDepDR'="" DEPDesc=$p($g(^RBC("DEP",CTLOCDepDR)),"^",2)  //科室部门组描述
    		continue:'$lf($lb("临床科室","护理单元"),DEPDesc)
    		s code = $p($g(^CTLOC(loc)),"^",1)
    		s query = ##class(%ResultSet).%New()
			s query.ClassName="web.DHCDocMain"
			s query.QueryName="FindOrd"

			if query.QueryIsValid() {
				
			    s rc = query.Execute(code)
			    continue:'rc
			    Set columns = query.GetColumnCount()
			    while (query.Next()) {
			        for i=1:1:columns {
            			s $list(Data,i)=query.GetData(i)
        			}
			        d OutputRow
			    }
			}
			d query.%Close()
		}
		Quit $$$OK
	}
	s code = $ZCONVERT(code, "U")
	
	s list = $lb(2030301002,
	2030301003,
	310205005,
	310205008,
	"体检39",
	310205009
	)
	; 在院 在床 ##Class(web.DHCNurseInterface).ifPatInBed()
	; ^PAADMi("CurrLoc",{PAADM_DepCode_DR},{PAADM_AdmDate},{PAADM_AdmTime},{PAADM_RowID})
	; ^PAADMi("CurrWard",{PAADM_CurrentWard_DR},+{PAADM_CurrentRoom_DR},{PAADM_RowID}
	s CTLocID = $o(^CTLOC(0,"Code",code,0))
	Quit:CTLocID="" $$$OK
	s CTLOCTYPE=$p($g(^CTLOC(CTLocID)),"^",13) //"科室类型"
    s DEPDesc=""
    s CTLOCDepDR=$p($g(^CTLOC(CTLocID)),"^",19)
    s:CTLOCDepDR'="" DEPDesc=$p($g(^RBC("DEP",CTLOCDepDR)),"^",2)  //科室部门组描述
    
	s locID = 0
	for {
		s locID = $O(^PAADMi("AdmTypeCurrLoc","I",locID))
		q:locID=""
		s Date = 0
		for {
			s Date = $O(^PAADMi("AdmTypeCurrLoc","I",locID,Date))
			q:Date=""
			s Time = 0
			for {
				s Time = $O(^PAADMi("AdmTypeCurrLoc","I",locID,Date,Time))
				q:Time=""
				s EpisodeId = 0
				
				for {
					s EpisodeId = $O(^PAADMi("AdmTypeCurrLoc","I",locID,Date,Time,EpisodeId))
					q:EpisodeId=""
					;continue:(EpisodeId'="792899")&&(EpisodeId'="792899")
					;b ;791724
					
					continue:##Class(web.DHCNurseInterface).ifPatInBed(EpisodeId)'="Y"
					continue:##class(web.UDHCJFPAY).CheckVisitStatus(EpisodeId)'="A"
					;b ;ifPatInBed
					s AdmWardDr=$p($g(^PAADM(EpisodeId)),"^",70)
					s WardCode = $p($g(^PAWARD(AdmWardDr)),"^",1)
					s LocId = $p(^PAADM(EpisodeId),"^",4)
					s Dept = $p($g(^CTLOC(LocId)),"^",2)
					s DeptCode = $p($g(^CTLOC(LocId)),"^",1)
					s pat = $p(^PAADM(EpisodeId),"^",1)
					s Name = $p(^PAPER(pat,"ALL"),"^")
					b:Name="陈刘妹"
					continue:(WardCode'=code)&&(DEPDesc="护理单元")
					continue:(DeptCode'=code)&&(DEPDesc="临床科室")
					;b ;EpisodeId
					d Episode
				}
			}
		}	
	}
	
	/*
	if DEPDesc="临床科室" {
		s date = 0
		for {
			s date = $o(^PAADMi("CurrLoc",CTLocID,date))
			q:date=""
			s time = 0
			for {
				s time = $o(^PAADMi("CurrLoc",CTLocID,date,time))
				q:time=""
				s EpisodeId = 0
				
				for {
					s EpisodeId = $o(^PAADMi("CurrLoc",CTLocID,date,time,EpisodeId))
					q:EpisodeId=""
					continue:##Class(web.DHCNurseInterface).ifPatInBed(EpisodeId)'="Y"
					continue:##class(web.UDHCJFPAY).CheckVisitStatus(EpisodeId)'="A"
					d Episode
				}	
			}
			
		}
	}
	*/
	
	
	/*
	病区编号	Varchar	No	若有科室无病区，可不提供。
病区名称	Varchar	No	
科室编号	Varchar	No	若有病区无科室，可不提供。
科室名称	Varchar	No	
病人编号	Varchar	No	患者在院唯一标识号（永久）
病人姓名	Varchar	No	
住院号	Varchar	No	
住院次数	Varchar		No	
床位名称	Varchar	No	床位号（1床、2床…）
医嘱编号	Varchar	No	
医嘱名称	Varchar	No	血糖医嘱、用药医嘱
医嘱标识	Varchar	No	区分治疗检验药品等
医嘱类别	Varchar	No	长期医嘱、临时医嘱
医嘱单位	Varchar	Yes	
医嘱用量	Varchar	Yes	
医嘱执行状态	Varchar	No	执行、停止、撤销等
开嘱医生	Varchar	No	
开始时间	datetime	No	医嘱开始时间
结束时间	datetime	Yes	医嘱结束时间
医嘱频次	Varchar	No	
医嘱嘱托	Varchar	Yes	
	*/
	Quit $$$OK

Episode
	s pat = $p(^PAADM(EpisodeId),"^",1)
	s Name = $p(^PAPER(pat,"ALL"),"^")
	s PatNo = $p(^PAPER(pat,"PAT",1),"^")
	s LocId = $p(^PAADM(EpisodeId),"^",4)
	s Dept = $p($g(^CTLOC(LocId)),"^",2)
	s DeptCode = $p($g(^CTLOC(LocId)),"^",1)
	s Ward = "", WardCode=""
	
	s AdmWardDr=$p($g(^PAADM(EpisodeId)),"^",70)
	if (AdmWardDr'="") {
		s WardCode = $p($g(^PAWARD(AdmWardDr)),"^",1)
		s Ward = $p($g(^PAWARD(AdmWardDr)),"^",2)
	}
	;b ;所在病区名称
	s AdmWardDesc=""
	if (AdmWardDr'="") s AdmWardDesc=$p($g(^PAWARD(AdmWardDr)),"^",2)
	s IPTimes=$P(^PAADM(EpisodeId),"^",29)
	s Bed = ##class(web.DHCDoc.OP.AjaxInterface).GetPatBedNo($g(EpisodeId))
	s IPNo = ##class(web.DHCDocOrderCommon).GetMrNo($g(EpisodeId),"","I","")
	Set ord = 0
	For {
		Set ord = $o(^OEORD(0,"Adm",EpisodeId,ord))
		Quit:ord=""
		Set item = 0
		For {
			Set item = $o(^OEORD(ord,"I",item))
            Quit:item=""
			s stat=$p($g(^OEORD(ord,"I",item,1)),"^",13)
			s:+stat>0 statCode = $p(^OEC("OSTAT",stat),"^",1)
			s:+stat>0 Stat = $p(^OEC("OSTAT",stat),"^",2)
			continue:(statCode'="E")&&(statCode'="V")
			
			
			s arcim = $p($g(^OEORD(ord,"I",item,1)),"^",2)
			s Code = $p($g(^ARCIM(+arcim,1,1)),"^",1)
			s Desc = $p($g(^ARCIM(+arcim,1,1)),"^",2)
			continue:'$lf(list,Code)
			;b ;code
			s Code = ord_"||"_item
			s Freq=""
			s FreqRowid=$P($G(^OEORD(ord,"I",item,2)),"^",4)
			s:FreqRowid'="" Freq=$p(^PHCFR(FreqRowid),"^",4)
			s PriorityDR = $p($g(^OEORD(ord,"I",item,1)),"^",8)
			s Prior = $p(^OECPR(PriorityDR),"^",2)
			s Starttime = $p($g(^OEORD(ord,"I",item,1)),"^",10)
			s Start = $p($g(^OEORD(ord,"I",item,1)),"^",9)
			;b ;Start
			s:Start'="" Start = $zd(Start,3)
			s:Starttime'="" Starttime=$zt(Starttime)
			s Start = Start_" "_Starttime
			s End = $p($g(^OEORD(ord,"I",item,3)),"^",34)
			s Endtime = $p($g(^OEORD(ord,"I",item,3)),"^",35)
			s:End'="" End = $zd(End,3)
			s:Endtime'="" Endtime = $zt(Endtime)
			s End = End_" "_Endtime
			s OrdPackQtyInfo=##class(web.DHCDocQryOEOrder).GetOrdPackQtyInfo(ord_"||"_item)
			s Qty = $p(OrdPackQtyInfo,"^",1)
			s Unit = $p(OrdPackQtyInfo,"^",2)
			
			s OrdCat = ""
			s Doctor=""
			s DoctorID=$p($g(^OEORD(ord,"I",item,1)),"^",11)
			s:$g(DoctorID)'="" Doctor=$p($g(^CTPCP(DoctorID,1)),"^",2)
			s Note=$g(^OEORD(ord,"I",item,"DEP",1))
			s Data = $lb(WardCode,Ward,DeptCode,Dept,PatNo,Name,IPNo,IPTimes,Bed, 
				Code,Desc,OrdCat,Prior,Unit,Qty,Stat,Doctor,Start,End,Freq,Note)
			
            d OutputRow
		}
	}
	q
OutputRow
	s ^CacheTemp(repid, ind) = Data
	s ind = ind + 1
	q
muse
	s $zt=""
	s ^muse=$lb($ze,$zdt($h,3))
	q $ze
}
